<img src="img/platform_introduction_native_app_header.png">

## Requirements
- Install the [Globus Python SDK](https://globus-sdk-python.readthedocs.io/en/stable/installation.html).
- Join the [Tutorial Users Group](https://app.globus.org/groups/50b6a29c-63ac-11e4-8062-22000ab68755) in order to experiment with sharing/guest collections.

## Globus SDK References
- Source code: https://github.com/globus/globus-sdk-python
- Documentation: https://globus-sdk-python.readthedocs.io/en/stable/

In [ ]:
import json  # just so we can pretty-print response data
import globus_sdk

# Feel free to replace the collection UUIDs below with those of your own collections
tutorial_collection_1 = "6c54cade-bde5-45c1-bdea-f4bd71dba2cc"  # collection "Globus Tutorial Collection 1"
tutorial_collection_2 = "31ce9ba0-176d-45a5-add3-f37d233ba47d"  # collection "Globus Tutorial Collection 2"
tutorial_users_group = "50b6a29c-63ac-11e4-8062-22000ab68755"  # group "Tutorial Users"

# This is the Tutorial GCS endpoint, which will be used for creating new Guest Collections
tutorial_endpoint_1 = "261692a4-7e49-4def-b59e-cbfc65e3907d"

CLIENT_ID = "3b1925c0-a87b-452b-a492-2c9921d3bd14"  # client ID of the Jupyter Demo App in Globus Auth
native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)

### Globus Scopes

Globus scopes are how applications request access to services. Scopes are used in OAuth2 Grants to obtain Access Tokens. This tutorial will use two scopes:

* Transfer Scope -- For access to the Globus Transfer API
* GCS Endpoint Scope -- For managing Guest Collections on Globus Tutorial Collection 1

Both of these scopes require access to data on collections, and so require the "data_access" scope dependency.

In [ ]:
# Get GCS Data Access scopes for both mapped collections. These are needed to access any data on
# a mapped collection
tc1_data_access = globus_sdk.scopes.GCSCollectionScopeBuilder(tutorial_collection_1).data_access
tc2_data_access = globus_sdk.scopes.GCSCollectionScopeBuilder(tutorial_collection_2).data_access

# Add scopes as dependencies for the transfer scope, allowing transfer operations to access data
# on both tutorial mapped collections.
transfer_scope = globus_sdk.scopes.TransferScopes.make_mutable("all")
transfer_scope.add_dependency(tc1_data_access)
transfer_scope.add_dependency(tc2_data_access)

# This scope will be used to manage the Endpoint on Tutorial 1, allowing us to Create/Update/Delete
# Guest Collections on this endpoint.
tutorial_ep1_manage_collections = globus_sdk.scopes.GCSEndpointScopeBuilder(tutorial_endpoint_1).make_mutable("manage_collections")
tutorial_ep1_manage_collections.add_dependency(tc1_data_access)

requested_scopes = [transfer_scope, tutorial_ep1_manage_collections]

### Globus Authorization Code Grant via JupyterHub

When you clicked "Login" when visiting jupyter.demo.globus.org, you were already asked to login with the scopes above. The tokens were then passed to this notebook. We can simply use those tokens instead of doing a Native Login Flow.

In [ ]:
import os
import base64
import pickle

USE_HUB_TOKENS = False
# The tokens are passed into the environment base64 encoded pickled Python dictionary 
# assigned to the GLOBUS_DATA variable. We'll grab the variable and unpack it.
globus_env_data = os.getenv('GLOBUS_DATA')  # get the raw base64 encoded data

if globus_env_data and USE_HUB_TOKENS is True:
    pickled_tokens = base64.b64decode(globus_env_data)  # get the pickled tokens
    # Unpickle and get the dictionary
    tokens = pickle.loads(pickled_tokens)["tokens"]
    print(json.dumps(tokens, indent=2))
else:
    print("Doing Native Auth instead of using Hub Tokens")

# Globus Native App Grant

The Native App Grant OAuth2 flow will do the following:
1. Log into Globus using a one-time, generated URL
2. Consent to allow this Jupyter Notebook to access Globus Transfer on our behalf
3. Return to the notebook with an Authorization Code (the result of step 2)
4. Exchange the Authorization Code for an Access Token
5. Create a TransferClient object using that Access Token as its authorization method

In [ ]:
if not globus_env_data or USE_HUB_TOKENS is False:
    # Explicitly start the flow (some clients may support multiple flows)
    native_auth_client.oauth2_start_flow(requested_scopes=requested_scopes, refresh_tokens=False)
    print(f"Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}")
    print("\nIMPORTANT NOTE: the link above can only be used once!")
    print("If login or a later step in the flow fails, you must execute this cell again to generate a new link.")
    auth_code = input("PASTE YOUR CODE HERE> ")
    tokens = native_auth_client.oauth2_exchange_code_for_tokens(auth_code).by_resource_server
    print("Tokens Received!")
else:
    print("Using 'Hub' Tokens instead.")

In [ ]:
# Extract the access token for the Globus Transfer service, known as "transfer.api.globus.org"
transfer_access_token = tokens['transfer.api.globus.org']['access_token']
tutorial_ep1_access_token = tokens[tutorial_endpoint_1]['access_token']

# Wrap the token in an object that implements the globus_sdk.GlobusAuthorizer interface
# In this case, an AccessTokenAuthorizer, which takes an access token and produces Bearer Auth headers
transfer_authorizer = globus_sdk.AccessTokenAuthorizer(transfer_access_token)
tutorial_ep1_authorizer = globus_sdk.AccessTokenAuthorizer(tutorial_ep1_access_token)

# Create a TransferClient object which Authorizes its calls using that GlobusAuthorizer
tc = globus_sdk.TransferClient(authorizer=transfer_authorizer)

## Help

Test that our transfer client works by requesting help on the `get_endpoint` method. You can use this to get help on any method.

In [ ]:
help(tc.get_endpoint)

## Using the client

The transfer client makes REST resources available via easy to use methods. The response from these methods wraps the HTTP response status, content type, text and JSON response body. 

In [ ]:
response = tc.get_endpoint(tutorial_collection_1)
print(f"HTTP Status Code: {response.http_status}")
print(f"Content Type: {response.content_type}")
print(f"Collection Display Name: {response['display_name']}")  # shortcut for response.data['display_name']
print(f"Data: {response}")

Helper methods for APIs that returns lists have iterable responses, and automatically take care of paging where required:

In [ ]:
response = tc.endpoint_search(filter_scope="my-endpoints")
for ep in response:
    print(f"{ep['display_name'] or ep['canonical_name']} ({ep['id']})")

If a helper method is not yet available for the desired API call, or for more low level control, the low-level interface can be used. Note that the low level interface does not do automatic paging:

In [ ]:
response = tc.get("/endpoint_search", query_params=dict(filter_fulltext="Globus Tutorial Collection", limit=1))
if response['DATA']:
    print(f"Endpoint ID: {response['DATA'][0]['id']} (Owner: {response['DATA'][0]['owner_string']})")
    print(f"More matches? {response['has_next_page']}")
else:
    print("No results")

## Handling errors

If the API returns an error response (HTTP status code 4xx or 5xx), it will be translated to a Python exception and raised:

In [ ]:
try:
    response = tc.get_endpoint("dcb2e10e-de27-4b99-8722-1a69aa3fc467")
except globus_sdk.GlobusAPIError as error:
    print(f"HTTP Status Code: {error.http_status}")
    print(f"Error Code      : {error.code}")
    print(f"Error Message   : {error.message}")

There are five basic classes of errors:

1. Bad request - there is something wrong with the request from the client, like a mispelled parameter name or missing required data. These errors have a code that starts with ``BadRequest`` or ``ClientError.BadRequest``.
2. State conflict - this is a very broad category, and covers all the errors that can happen during normal operation, and neither the client nor the server could have anticipated and avoided the error. Examples: local filesystem permissions not allowing the requested path on a remote GridFTP endpoint, endpoint not found (could have been deleted concurrently by another client). This also includes network errors communicating with GridFTP endpoints and other external services. These errors typically have a code containing ``PermissionDenied``, ``Conflict``, or ``ExternalError``.
3. Network error - network failure between the REST client and the REST API server. These errors will result in a ``globus_sdk.NetworkError`` being raised by the SDK.
4. Planned downtime - code ``ServiceUnavailable``.
5. Server error - caused by a bug in the REST API server (code ``ServerError.InternalError``). We log such errors and incorperate fixes into our next release, but developers are still encouraged to submit details to the mailing list when they encounter these errors. Note that sometimes these errors are actually a sign of a bad request type error, i.e. the bug in the server is that it's not anticipating the exact type of bad data, so it's not reporting the correct error code, but the problem can still be resolved by a change to the client.

# Collection Management

## Collection search

Globus has tens of thousands of registered collections. To find collections of interest you can access powerful search capabilities via the SDK. For example, to search for a given string across the descriptive fields of collections (names, description, keywords):

In [ ]:
search_str = "Globus Tutorial Collection"
collections = tc.endpoint_search(search_str)
print(f"==== Displaying collections that match '{search_str}' ===")
for col in collections:
    print(f"{col['display_name'] or col['canonical_name']} ({col['id']})")

## Restricting search scope with filters

There are also a number of default filters to restrict the search for 'my-endpoints', 'my-gcp-endpoints',     'recently-used', 'in-use', 'shared-by-me','shared-with-me') 

In [ ]:
search_str = None
collections = tc.endpoint_search(filter_fulltext=search_str, filter_scope="recently-used")
for col in collections:
    print(f"{col['display_name'] or col['canonical_name']} ({col['id']})")

## Collection details

You can also retrieve complete information about an collection, including name, owner, and configuration details. 

In [ ]:
collection = tc.get_endpoint(tutorial_collection_1)
print(f"Display Name: {collection['display_name']}")
print(f"Owner       : {collection['owner_string']}")
print(f"ID          : {collection['id']}")
print(f"Network Use - Concurrency = {collection['preferred_concurrency']}")
print(f"Network Use - Parallelism = {collection['preferred_parallelism']}")

# File operations

## Get a directory listing

Having activated an collection, you can now perform operations on it. For example, performing an ls command to retrieve directory contents. 

In [ ]:
# help(tc.operation_ls)
collection_id = tutorial_collection_1
collection_path = "/home/share/godata/"
response = tc.operation_ls(collection_id, path=collection_path)
print(f"==== 'ls' for {collection_path} on collection {collection_id} ====")
for item in response:
    print(f"{item['type']}: {item['name']} [{item['size']}]")

## Make directory

You can create a new directory.

In [ ]:
# help(tc.operation_mkdir)
try:
    new_path = "/~/tutorial_dir"
    mkdir_result = tc.operation_mkdir(collection_id, path=new_path)
    print(mkdir_result['message'])
except globus_sdk.GlobusAPIError as error:
    print(f"Error code: {error.code}\nError message: {error.message}")

## Rename

You can rename files and directories on your collections. 

In [ ]:
# help(tc.operation_rename)
try:
    response = tc.operation_rename(collection_id, oldpath="/~/tutorial_dir", newpath="/~/tutorial_dir_renamed")
    print(response['message'])
except globus_sdk.GlobusAPIError as error:
    print(f"Error code: {error.code}\nError message: {error.message}")

# Task submission and management

The Globus task interface allows you to create and manage asynchronous file transfer and deletion tasks. 

## Transfer

Creating a transfer is a two stage process. First you must create a description of the data you want to transfer (which also creates a unique submission_id), and then you can submit the request to Globus to transfer that data. 

If the submit_transfer fails, you can safely resubmit the same transfer_data again. The submission_id will ensure that this transfer request will be submitted once and only once.

In [ ]:
# help(tc.submit_transfer)
source_collection_id = tutorial_collection_1
source_path = "/home/share/godata/"

dest_collection_id = tutorial_collection_2
dest_path = "/~/"

label = "My tutorial transfer"

# TransferData() automatically gets a submission_id for once-and-only-once submission
tdata = globus_sdk.TransferData(tc, source_collection_id, dest_collection_id, label=label)

## Recursively transfer source path contents
tdata.add_item(source_path, dest_path, recursive=True)

## Alternatively, transfer a specific file
# tdata.add_item("/source/path/file.txt", "/dest/path/file.txt"))

submit_result = tc.submit_transfer(tdata)
print(f"Task ID: {submit_result['task_id']}")

## Get Task By ID

While the task is running, or after completion, you can get information that describes the transfer task. 

In [ ]:
response = tc.get_task(submit_result['task_id'])
print(f"Label: {response['label']}")
print(f"Status: {response['status']}")
print(f"Transfer: {response['source_endpoint_display_name']} -> {response['destination_endpoint_display_name']}")
    
if response.data["status"] == "SUCCEEDED":
    print(f"Bytes transferred: {response['bytes_transferred']}")
    print(f"Files transferred: {response['files_transferred']}")
    print(f"Transfer rate: {response['effective_bytes_per_second']} Bps")

## Check destination collection

After the transfer has finished you can list the contents of the destination collection

In [ ]:
ls_iter = tc.operation_ls(dest_collection_id, path=dest_path)
print(f"==== 'ls' for {dest_path} on collection {dest_collection_id} ====")
for item in ls_iter:
    print(f"{item['type']}: {item['name']} [{item['size']}]")

## Get task list

You can get a list of past or current tasks with the following call. Note that only `TRANSFER` tasks are returned by default - the type filter is necessary to get `DELETE` tasks as well. This is a remnant of a legacy backward compatibility concern, and will likely be changed in the future to both being included by default.

In [ ]:
# help(tc.task_list)
response = tc.task_list(limit=10, filter="type:TRANSFER,DELETE")
for i, item in enumerate(response):
    print(item['status'],
          item['task_id'], 
          item['type'],
          item['source_endpoint_display_name'],
          item['destination_endpoint_display_name'],
          item['label'])

## Filter task list

Retrieve only active tasks:

In [ ]:
response = tc.task_list(limit=10, filter="type:TRANSFER,DELETE/status:ACTIVE")
for i, item in enumerate(response):
    print(item['status'],
          item['task_id'], 
          item['type'],
          item['source_endpoint_display_name'],
          item['destination_endpoint_display_name'],
          item['label'])

See [Common Query Parameters](https://docs.globus.org/api/transfer/task/#common_query_parameters) for a description of the `filter` parameter, and [Task List filters](https://docs.globus.org/api/transfer/task/#filter_and_order_by_options) for details of what is supported by task list.

## Cancel task

You can also cancel a running task. 

In [ ]:
# help(tc.cancel_task)
response = tc.cancel_task(submit_result['task_id'])
print(f"{response['code']}: {response['message']}")

## Get event list for task

Every task stores periodic event markers (e.g., errors, performance markers, etc.). You can retrieve and filter this list as follows. 

In [ ]:
# help(tc.task_event_list)
response = tc.task_event_list(submit_result['task_id'], limit=10)
for event in response:
    print(event['time'], event['code'], event['is_error'], event['details'])


## Delete files task

File deletion is also an asynchronous task, that is submitted and monitored similar to a transfer task.

In [ ]:
# help(tc.submit_delete)
# Create a folder, delete it, wait for completion
collection_id = tutorial_collection_1
path = "/~/tutorial_delete_example"
try:
    tc.operation_mkdir(collection_id, path=path)
except globus_sdk.GlobusAPIError as error:
    if "Exists" in error.code:
        print("Directory already exists, ignoring error")
    else:
        raise

label = "My tutorial delete"

# DeleteData() automatically gets a submission_id for once-and-only-once submission
# Note that recursive is a top level option for delete, not a per-path option like
# it is for transfers.
ddata = globus_sdk.DeleteData(tc, collection_id, label=label, recursive=True)

## Recursively delete path contents (because of recursive flag set above)
ddata.add_item(path)

submit_result = tc.submit_delete(ddata)
print(f"Task ID: {submit_result['task_id']}")

## Wait for task to complete

Transfer and delete tasks are asynchronous operations, and depending on their size may take a long time to complete. If you wish to wait for a task to complete, the TransferClient provides a task_wait helper method:

In [ ]:
# Wait for a task to finish for 10 minutes, polling every 15 seconds.
completed = tc.task_wait(submit_result['task_id'], timeout=600, polling_interval=15)
if completed:
    print("Task finished!")
else:
    print("Task still running after timeout reached.")


# Bookmarks

Bookmarks allow you to keep a list of frequently used collections and paths. Full management capabilities (create, retrieve, update, delete) are supported on bookmarks. Note that the REST API itself does not directly support bookmarks when performing operations. It is the responsibility of the client to allow the users to choose bookmarks, and then translate them to collection ids to perform ls operations and submit transfers. In particular, the www.globus.org website has full support for bookmarks.

## Create a Bookmark

In [ ]:
bookmark_name = "My Tutorial Bookmark"
collection_id = tutorial_collection_1
collection_path = "/home/share/godata/"
response = tc.create_bookmark({"endpoint_id": collection_id, "path": collection_path,"name": bookmark_name})
bookmark_id = response['id']
print(response)

## Get a list of bookmarks

In [ ]:
response = tc.bookmark_list()
for b in response:
    print (b['name'], b['path'], b['id'])

## Update a bookmark


In [ ]:
bookmark_data = {
    'name': 'My Updated Tutorial Bookmark'
}
response = tc.update_bookmark(bookmark_id, bookmark_data)
print (response)

## Delete a Bookmark

In [ ]:
response = tc.delete_bookmark(bookmark_id)
print (response)

# Guest Collections

Guest Collections are virtual collections that refer to a particular "host guest collection" and path, which allows Globus to manage access to that mapped collection. Folders on the guest collection can be easily shared with other Globus users and groups via access control rules.

## Create a guest collection

In [ ]:
# Fetch the URL for the tutorial endpoint, so we can Query the GCS Manager
tutorial_collection_info = tc.get_endpoint(tutorial_endpoint_1)
gcs_manager_url = tutorial_collection_info["gcs_manager_url"]

# Fetch the local credential information from the GCS Manager
gc = globus_sdk.GCSClient(tutorial_collection_info["gcs_manager_url"], authorizer=tutorial_ep1_authorizer)
local_ep1_user_credential = gc.get(path="/api/user_credentials").data["data"][0]
print(f"Found local user credential with username {local_ep1_user_credential['username']}")

In [ ]:
# Create a dir to share
host_collection_id = tutorial_collection_1
host_collection_path = "/~/shared_dir2"
try:
    response = tc.operation_mkdir(host_collection_id, path=host_collection_path)
except globus_sdk.GlobusAPIError as error:
    # Ignore the error if the directory already exists, otherwise raise
    if "Exists" not in error.code:
        raise

# Define the guest collection
guest_collection = {
        "DATA_TYPE": "collection#1.4.0",
        "collection_type": "guest",
        "display_name": "My Tutorial Guest Collection2",
        "description": "Test creating a share from globus-jupyter-notebook",
        "identity_id": local_ep1_user_credential["identity_id"],
        "storage_gateway_id": local_ep1_user_credential["storage_gateway_id"],
        "collection_base_path": host_collection_path,
        "public": True,
        "user_credential_id": local_ep1_user_credential["id"],
        "mapped_collection_id": host_collection_id,
}

response = gc.post("/api/collections/", data=guest_collection)
print(f"{response['code']}: {response['message']}")
guest_collection_id = response["data"][0]["id"]

## Get collection information

In [ ]:
response = tc.get_endpoint(guest_collection_id)
print(f"Display name: {response['display_name']}")
print(f"Owner: {response['owner_string']}")
print(f"Host Endpoint ID: {response['host_endpoint_id']}")

## Get a list of guest collections

In [ ]:
endpoints = tc.endpoint_search(filter_scope="shared-by-me")
print("==== Displaying guest collections ===")
for ep in endpoints:
    print(f"{ep['display_name']} ({ep['id']})")


## Add a new access control rule

You can share access to different paths within your guest collection with users, groups, or publicly. The principal_type can be one of 'identity', 'group', 'all_authenticated_users', or 'anonymous'.  Each access rule is given a unique access_rule_id, which can be used to manage that access rule.

Here is an example of sharing with the tutorial users group. 

In [ ]:
rule_data = {
    'DATA_TYPE': 'access',
    'permissions': 'rw',
    'principal' : tutorial_users_group,  # use this if sharing with a group of users
    'principal_type' : 'group',  # use this if sharing with a group of users
    #'principal': 'IDENTITY_ID',  # use this if sharing with a single user (identity)
    #'principal_type': 'identity',  # use this if sharing with a single user (identity)
    'path': '/'
}

try:
    response = tc.add_endpoint_acl_rule(guest_collection_id, rule_data)
    access_rule_id = response['access_id']
    print (response)
except globus_sdk.GlobusAPIError as error:
    if "Exists" in error.code:
        print("ACL already exists, ignoring error")
    else:
        raise

## Get list of access rules

In [ ]:
response = tc.endpoint_acl_list(guest_collection_id)
for rule in response:
    print (rule['id'], rule['principal_type'], rule['principal'], rule['permissions'], rule['path'])
print(response)

## Get access rule by id

Get the access rule details using its access_rule_id

In [ ]:
response = tc.get_endpoint_acl_rule(guest_collection_id, access_rule_id)
print (response)

## Update access rule

Update an access rule using its access_rule_id.

In [ ]:
rule_update = {
    'DATA_TYPE': 'access',
    'permissions': 'r',
}
response = tc.update_endpoint_acl_rule(guest_collection_id, access_rule_id, rule_update)
print (response)

## Delete access rule

Delete an access rule using its access_rule_id.

In [ ]:
response = tc.delete_endpoint_acl_rule(guest_collection_id, access_rule_id)
print (response)

# Low-level SDK interface

The helper methods are all built on top of the low level interface. If a helper method is not yet available for the API resource you wish to use, the low level interface can be used directly.

Note that the examples in this section use the endpoint management API resources, which DO have helper methods, but they still serve as good examples for how to use the low level interface.

## POST request

Create an endpoint using the low level API:

In [ ]:
guest_collection = {
        "DATA_TYPE": "collection#1.4.0",
        "collection_type": "guest",
        "display_name": "Tutorial Create Example",
        "description": "Test creating a share from globus-jupyter-notebook",
        "identity_id": local_ep1_user_credential["identity_id"],
        "storage_gateway_id": local_ep1_user_credential["storage_gateway_id"],
        "collection_base_path": host_collection_path,
        "public": True,
        "user_credential_id": local_ep1_user_credential["id"],
        "mapped_collection_id": host_collection_id,
}

response = gc.post("/api/collections/", data=guest_collection)
guest_collection_id = response["data"][0]["id"]
print(response)

## GET request

Do a GET on the newly create endpoint:

In [ ]:
response = gc.get(f"/api/collections/{guest_collection_id}",
           query_params=dict(fields="id,display_name,description"))
print(response)

## PUT request

Update the description on the newly created endpoint:

In [ ]:
endpoint_update = {
    "DATA_TYPE": "collection#1.4.0",
    "public": True,
    "collection_type": "guest",
    "display_name": "Tutorial Update Example",
    "collection_base_path": host_collection_path,
    "description": "Test updating description using low level API",
    "identity_id": local_ep1_user_credential["identity_id"],
    "storage_gateway_id": local_ep1_user_credential["storage_gateway_id"],
    "user_credential_id": local_ep1_user_credential["id"],
    "mapped_collection_id": host_collection_id,
}
response = gc.put(f"/api/collections/{guest_collection_id}", data=endpoint_update)
print(response)

## DELETE request

Now delete the endpoint:

In [ ]:
response = gc.delete(f"/api/collections/{guest_collection_id}")
print(response)

## More exercises

You can practice working with the SDK and the Globus Transfer service with [the exercises in this notebook](Transfer_API_Exercises.ipynb).